# Estimating Spot and Roll Returns Using the Constant Returns Model

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Load Data

In [2]:
F_BRdf = pd.read_csv('data/F_BR_history.csv', skiprows=1)
F_BRdf.columns = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
F_BRdf['Date'] = pd.to_datetime(F_BRdf['Date'], errors='coerce')
F_BRdf['Price'] = pd.to_numeric(F_BRdf['Price'], errors='coerce')

F_CLdf = pd.read_csv('data/F_CL_history.csv', skiprows=1)
F_CLdf.columns = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
F_CLdf['Date'] = pd.to_datetime(F_CLdf['Date'], errors='coerce')
F_CLdf['Price'] = pd.to_numeric(F_CLdf['Price'], errors='coerce')

F_HGdf = pd.read_csv('data/F_HG_history.csv', skiprows=1)
F_HGdf.columns = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
F_HGdf['Date'] = pd.to_datetime(F_HGdf['Date'], errors='coerce')
F_HGdf['Price'] = pd.to_numeric(F_HGdf['Price'], errors='coerce')

F_HGdf.head()

F_ZCdf = pd.read_csv('data/F_ZC_history.csv', skiprows=1)
F_ZCdf.columns = ['Date', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change %']
F_ZCdf['Date'] = pd.to_datetime(F_ZCdf['Date'], errors='coerce')
F_ZCdf['Price'] = pd.to_numeric(F_ZCdf['Price'], errors='coerce')

F_HGdf.head()

,Date,Price,Open,High,Low,Vol,Change %
0,2023-10-18,3.5870,3.5900,3.6305,3.5750,69.58K,0.24%
1,2023-10-17,3.5785,3.5815,3.5930,3.5315,86.04K,-0.10%
2,2023-10-16,3.5820,3.5705,3.6055,3.5635,57.58K,0.31%
3,2023-10-13,3.5710,3.5870,3.6165,3.5570,64.95K,-0.56%
4,2023-10-12,3.5910,3.6115,3.6485,3.5695,69.71K,-0.58%


## Spot Returns Estimation

In [3]:
def estimate_spot_return(spot_prices):
    T = np.arange(len(spot_prices))
    T = sm.add_constant(T)
    model = sm.OLS(np.log(spot_prices), T)
    results = model.fit()
    return 252 * results.params[1]


## Roll Returns Estimation

In [4]:
def estimate_roll_return(futures_data):
    gamma = []
    for t in range(len(futures_data)):
        FT = futures_data.iloc[t]
        valid_indices = np.where(~np.isnan(FT))[0]
        consecutive_check = np.diff(valid_indices) == 1
        
        if len(valid_indices) >= 5 and all(consecutive_check[:4]):  
            FT_subset = FT[valid_indices[:5]].values
            T_subset = np.arange(len(FT_subset)).reshape(-1, 1)
            T_subset = sm.add_constant(T_subset)
            model = sm.OLS(np.log(FT_subset), T_subset)
            results = model.fit()
            gamma.append(-12 * results.params[1])
        else:
            gamma.append(np.nan)
    return np.nanmean(gamma)